In [3]:
with open('file1.txt') as f :
    for row in f:
        print(row,end='')
print('\n-----------------')
with open('file2.txt') as f :
    for row in f:
        print(row,end='')
print('\n-----------------')
with open('file2.txt') as f :
    for row in f:
        print(row,end='')
print('\n-----------------')

file1_line1
file1_line2
file1_line3
-----------------
file2_line1
file2_line2
file2_line3
-----------------
file2_line1
file2_line2
file2_line3
-----------------


In [4]:
with open('file1.txt')as f1,open('file2.txt')as f2:
    print(f1.readlines())
    print(f2.readlines())

['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']


In [5]:
with open('file1.txt')as f1:
    with open('file2.txt')as f2:
        with open('file3.txt')as f3:
            print(f1.readlines())
            print(f2.readlines())
            print(f3.readlines())

['file1_line1\n', 'file1_line2\n', 'file1_line3']
['file2_line1\n', 'file2_line2\n', 'file2_line3']
['file3_line1\n', 'file3_line2\n', 'file3_line3']


In [6]:
from contextlib import contextmanager

In [8]:
@contextmanager
def open_file(f_name):
    print(f'opening {f_name}')
    f = open(f_name)
    try:
        yield f
    finally:
        print(f'closing{f_name}')
        f.close()

In [10]:
f_names ='file1.txt','file2.txt','file3.txt'
exits = []
enters = []

for f_name in f_names:
    ctx = open_file(f_name)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)

In [11]:
files = [enter() for enter in enters]


opening file1.txt
opening file2.txt
opening file3.txt


In [12]:
while True:
    try:
        rows = [next(f).strip('\n') for f in files]
    except StopIteration:
        break
    else:
        row = ','.join(rows)
        print(row)

file1_line1,file2_line1,file3_line1
file1_line2,file2_line2,file3_line2
file1_line3,file2_line3,file3_line3


In [14]:
for exit in exits[::-1]:
    exit(None,None,None)

closingfile3.txt
closingfile2.txt
closingfile1.txt


In [15]:
f_names ='file1.txt','file2.txt','file3.txt'
#creating context managers
exits = []
enters = []

for f_name in f_names:
    ctx = open_file(f_name)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)

#entering context managers
files = [enter() for enter in enters]

# do work
while True:
    try:
        rows = [next(f).strip('\n') for f in files]
    except StopIteration:
        break
    else:
        row = ','.join(rows)
        print(row)

# exiting context managers
for exit in exits[::-1]:
    exit(None,None,None)

opening file1.txt
opening file2.txt
opening file3.txt
file1_line1,file2_line1,file3_line1
file1_line2,file2_line2,file3_line2
file1_line3,file2_line3,file3_line3
closingfile3.txt
closingfile2.txt
closingfile1.txt


In [24]:
class NestedContexts:
    def __init__(self,*contexts):
        self._enters = []
        self._exits = []
        self._values = []
        
        for ctx in contexts:
            self._enters.append(ctx.__enter__)
            self._exits.append(ctx.__exit__)
            
    
    def __enter__(self):
        for enter in self._enters:
            self._values.append(enter())
        return self._values
    
    def __exit__(self,exc_type,exc_value,exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type,exc_value,exc_tb)
        return False
    

In [26]:
with NestedContexts(open_file('file1.txt'),
                   open_file('file2.txt'),
                   open_file('file3.txt'))as files:
    while True:
        try:
            rows = [next(f).strip('\n') for f in files]
        except StopIteration:
            break
        else:
            row = ','.join(rows)
            print(row)

opening file1.txt
opening file2.txt
opening file3.txt
file1_line1,file2_line1,file3_line1
file1_line2,file2_line2,file3_line2
file1_line3,file2_line3,file3_line3
closingfile3.txt
closingfile2.txt
closingfile1.txt


In [28]:
f_names ='file1.txt','file2.txt','file3.txt'

contexts = [open_file(f_name) for f_name in f_names]
with NestedContexts(*contexts) as files:
    print('do work')

opening file1.txt
opening file2.txt
opening file3.txt
do work
closingfile3.txt
closingfile2.txt
closingfile1.txt


In [29]:
class NestedContexts:
    def __init__(self):
        self._exits = []
        
    def __enter__(self):
        return self
    
    def enter_context(self,ctx):
        self._exits.append(ctx.__exit__)
        value = ctx.__enter__()
        return value
    
    def __exit__(self,exc_type,exc_value,exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type,exc_value,exc_tb)
        return False
    

In [31]:
f_names ='file1.txt','file2.txt','file3.txt'

with NestedContexts()as stack:
    files = [stack.enter_context( open_file(f)) for f in f_names]
    
    #do work
    print('do work')

opening file1.txt
opening file2.txt
opening file3.txt
do work
closingfile3.txt
closingfile2.txt
closingfile1.txt


In [32]:
from contextlib import ExitStack

In [34]:
f_names ='file1.txt','file2.txt','file3.txt'

with ExitStack() as stack:
    files = [stack.enter_context( open_file(f)) for f in f_names]
    print('do work')

opening file1.txt
opening file2.txt
opening file3.txt
do work
closingfile3.txt
closingfile2.txt
closingfile1.txt
